# 確率ロボティクス2017第5回

上田隆一

2017年10月18日@千葉工業大学

## 今日やること

* リサンプリング
* ベイズフィルタ（理論）

## リサンプリング

* パーティクルの重みが偏ると近似の効率が悪くなる
  * 是正するための処理
* よく似た手続き: 選挙における議席の調整
  * 各県の人口が偏る$\rightarrow$議席の数を調整
    * 人口の多い議員が多く票を持っているというシステムは、たぶんおかしなことが起こるのでどの国もとっていない。（株式会社はその逆。大株主がおかしくなると会社は潰れる。）
  * 「サンプリング（標本抽出）」

## 数式上の説明

* 問題
  * ある時点でのパーティクルの集合$\Xi = \{\xi^{(i)} = (\boldsymbol{x}^{(i)}, w^{(i)}) | i = 1,2,\dots,N \}$の$w^{(i)}$が大きくばらついている
  * このパーティクルの集合は、$Bel(X) = \int_{\boldsymbol{x} \in X} bel(\boldsymbol{x}) d\boldsymbol{x} \approx \sum_{i=1}^N \delta(\boldsymbol{x}^{(i)} \in X) w^{(i)}$を近似している
  * **近似精度を保ったまま、パーティクルの重みがすべて等しく$1/N$となるパーティクルの集合を新たに作るにはどうしたらよいか？**

## 主な（リ）サンプリングの手法

* パーティクルフィルタで使って意味のあるもの
  * 単純ランダムサンプリング
  * 層化抽出法（stratified sampling, 層別サンプリング）
  * **系統抽出（systematic sampling, 系統サンプリング、等間隔サンプリング）**
  * **KLDサンプリング**
    * これがROSのMCL（amcl）で用いられているのですが、すみません、扱いません・・・
* 他、商品の抜き取り検査の解説ページ等に豊富な説明が存在

 ## 系統抽出法の実装
 
* 単純な例で
  * パーティクルは4つ
  * 重みは次の通りで、リストにしましょう。

In [1]:
%matplotlib inline
import numpy as np
import math, random, copy
import matplotlib.pyplot as plt                   #   for plotting data

class Particle:
    def __init__(self,p,w ):  #姿勢も指定できるようにしましょう
        self.pose = np.array(p)
        self.weight = w
    
    def __repr__(self):
        return "pose: " + str(self.pose) + " weight: " + str(self.weight)

particles = [Particle([1.0,0.0,0.0],0.1),  #1個目のパーティクル
            Particle([2.0,0.0,0.0],0.2),   #2個目のパーティクル
            Particle([3.0,0.0,0.0],0.3),   #3個目のパーティクル
            Particle([4.0,0.0,0.0],0.4)]   #4個目のパーティクル

In [2]:
for p in particles:
    print(p)

pose: [ 1.  0.  0.] weight: 0.1
pose: [ 2.  0.  0.] weight: 0.2
pose: [ 3.  0.  0.] weight: 0.3
pose: [ 4.  0.  0.] weight: 0.4


## （累積）確率分布のリストを作成

In [3]:
accum = []
sm = 0.0
for p in particles:
    accum.append(p.weight + sm)
    sm += p.weight
    
print(accum)

[0.1, 0.30000000000000004, 0.6000000000000001, 1.0]


## サンプリング

* $[0,1/N)$の間の数（ポインタ）を乱数で一つ選択
* ポインタの数が1未満の間、以下、繰り返し
  * リストの先頭の確率がポインタの数以上
    * リストに対応するパーティクルの姿勢と重み$1/N$で、新たにパーティクルを生成
    * ポインタの数字を$1/N$増加
  * 未満
    * 今のリストの先頭を消す

In [4]:
pointer = random.uniform(0.0,1.0/len(particles))
print(pointer)

0.18238983434829664


In [5]:
new_particles = []
particles_num = len(particles)

while pointer < 1.0:
    if accum[0] >= pointer:
        new_particles.append(
            Particle(copy.deepcopy(particles[0].pose),1.0/particles_num)
        )
        pointer += 1.0/particles_num
    else:
        accum.pop(0)
        particles.pop(0)
        

In [6]:
particles = new_particles

for p in particles:
    print(p)

pose: [ 2.  0.  0.] weight: 0.25
pose: [ 3.  0.  0.] weight: 0.25
pose: [ 4.  0.  0.] weight: 0.25
pose: [ 4.  0.  0.] weight: 0.25


余裕のある人はパーティクルの数を増やしてみましょう。

## 課題1

* MCLの実装
  * 2-5回のコードをうまく組み合わせて、パーティクルフィルタによる自己位置推定（Monte Carlo localization）のコードを書く
    * motion update
    * sensor update
    * resampling
      * sensor updateの直後に適用すると良い
* 提出方法
  * GitHubにJupyter notebookを置く
  * 上田までURLをメール
  * メール
    * 件名: 確率ロボティクス2017年課題1
    * 内容に氏名、学籍番号を書いてください
* 期限
  